# D&D 5e Abstract Syntax Tree

The goal of this library is to create an abstract syntax tree capable of representing the mechanics of 5th edition D&D.

## Nodes

### Node types

* **Roll.** Randomly determines an outcome or returns a distribution of outcomes based on their probabilities. 
* **Selection.** Converts an outcome to a value.
* **Value.** Aren't interpreted. Return only their assigned value.
* **Effect.** Anything that affects a creature or target.
* **Control.** Relate values to one another. If statements.
* **Target.** Used to determine one or more targets.
* **Duration.** Specifies how long and effect lasts and under what circumstances it ends.

### Dice Roll

These nodes are simple dice equations that can be evaluated for a variety of uses.

```python
{
    'node': 'Roll',
    'equation': '1d20 + 5',
}
```

### D20 Test

These nodes are used to select an outcome following a d20 roll. There are three subtypes: attacks, saves, and checks. In all cases, a d20 roll is made, modifiers are added to the roll, and one of several outcomes occurs depending on the roll.

 * **Attacks.** Miss, hit, or critical hit.
 * **Saves.** Success, failure.
 * **Checks.** Success, failure.

Perhaps something to this effect.

```python
{
    'node': 'D20Test',
    'type': 'Attack',
    'modifiers': ['+4'],
    'results': [
        ('Critical Miss', '{d20}=1', EffectNode()),
        ('Critical Hit', '{d20}=20', EffectNode()),
        ('Miss', '{roll}<{target_AC}', EffectNode()),
        ('Hit', '{roll}>={target_AC}', EffectNode()),
    ],
}
```

```python
{
    'node': 'D20Test',
    'type': 'Save',
    'DC': '14',
    'results': [
        ('Success', '{roll}>={DC}', EffectNode()),
        ('Failure', '{roll}<{DC}', EffectNode()),
    ],
}
```

```python
{
    'node': 'D20Test',
    'type': 'Check',
    'modifiers': ['+4'],
    'results': [
        ('Success', '{roll}>={DC}', EffectNode()),
        ('Failure', '{roll}<{DC}', EffectNode()),
    ],
}
```

Alternatively, could have a variables parameter that can be used to associate different values and 
rolls to a name that can be referenced in equations in the results field. 

For example,

```python
{
    'node': 'D20Test',
    'type': 'Attack',
    'variables': [
        ('d20', RollNode('1d20')),
        ('mod', NumberNode('4')),
    ],
    'results': [
        ('Critical Miss', '{d20}=1', EffectNode()),
        ('Critical Hit', '{d20}=20', EffectNode()),
        ('Miss', '{d20}+{mod}<{target_AC}', EffectNode()),
        ('Hit', '{d20}+{mod}>={target_AC}', EffectNode()),
    ],
}
```

Another alternative, instead of equations in the results list, a function could be used to determine the results. This function would need to be passed into the evaluator. This might cause issues with wanting to store the AST as a dictionary or json file.

```python
{
    'node': 'D20Test',
    'type': 'Attack',
    'evaluator': 'attack_evaluator',
    'variables': {
        'critical_hit_range': [19,20],
        'critical_miss_range': [1],
        'attack_bonus': 4,
    },
    'results': {
        'critical miss': EffectNode(),
        'miss': EffectNode(),
        'hit': EffectNode(),
        'critical hit': EffectNode(),
    }
}
```

and for saving throws

```python
{
    'node': 'D20Test',
    'type': 'Save',
    'evaluator': 'save_evaluator',
    'variables': {
        'save_dc': 12,
    },
    'results': {
        'Success': EffectNode(),
        'Failure': EffectNode(),
    }
}
```

What about having different types of roll nodes. Like an AttackRoll.

```python
{
    'node': 'AttackRoll',
    'critical_hit_range': [19,20],
    'critical_miss_range': [1],
    'attack_bonus': 5,
    'armor_class': 'target.armor_class',
}
```

```python
{
    'node': 'SaveRoll',
    'save_dc': 13,
    'save_bonus': 'target.save_bonus',
}
```

and then a selection node

```python
{
    'node': 'Selection',
    'selector': AttackRollNode(),
    'results': {
        'critical miss': EffectNode(),
        'miss': EffectNode(),
        'hit': EffectNode(),
        'critical hit': EffectNode(),
    },
}
```

### Effects

These nodes alter the target in some way. Effect nodes have the following subtypes:

* Healing
* Damage
* Conditions
* Forced Movement
* Bonuses
* Penalties


An effect node should could take this form,

```python
{
    'node': 'Effect',
    'timing': TimingNode(),  # when to evaluate the effect (start of a turn, now, etc.)
    'duration': DurationNode(), # how long the effect lasts, or when it ends (instantaneous, permanent, 1 minute, etc.)
    'results': [], # what the effect does (damage, healing, apply a condition)
}
```

```python
{
    'node': 'Effect',
    'type': 'Healing',
    'timing': TimingNode(),
    'duration': DurationNode(),
    'results': RollNode(),
}
```

```python
{
    'node': 'Effect',
    'type': 'Damage',
    'timing': TimingNode(),  
    'duration': DurationNode(), 
    'results': (RollNode(), 'damage type'), 
}
```

```python
{
    'node': 'Effect',
    'type': 'Condition',
    'timing': TimingNode(),  
    'duration': DurationNode(), 
    'results': 'condition name', 
}
```

### Durations

These nodes are used to define how long an effect lasts, or, more precisely, when it ends. This should cover

* lengths of time
* concentration
* conditional triggers
* saves
* actions

```python
{
    'node': 'Duration',
    'ends': [
        '1 minute',
        'caster.concentration != self',
    ],
}
```

### Target Selectors

These nodes tell us how to pick targets using any of the following criteria

* range
* area
* number
* visibility

Areas can take any of the following shapes:
 * **Cone.** length/width
 * **Cube.** edge length
 * **Cylinder.** radius/diameter, height
 * **Emanation.** radius/diameter
 * **Line.** length and width
 * **Sphere.** radius/diameter

```python
{
    'node': 'Targeting',
    'range': '60 feet',
    'area': {
        'shape': 'cylinder',
        'radius': '20 feet',
        'height': '40 feet',
    },
    'max_number': 2,
    'min_number': 0,
}
```

This would work in tandem with an array of targets, who would be selected based on their attributes.

Instead of passing a single target dictionary to the evaluator, a dictionary of possible targets and parameters could be used.
For example:

```python
targets = {
    'maxtargets': 5,
    'melee_maxtargets': 2,
    'melee_target': {
        'armor_class': 12,
        'strength_save_bonus': 1,
        'dexterity_save_bonus': 0,
        'constitution_save_bonus': 2,
        'intelligence_save_bonus': 1,
        'wisdom_save_bonus': 2,
        'charisma_save_bonus': 2,
    },
    'ranged_targetarea': 10**2,
    'ranged_maxtargets': 4,
    'ranged_target': {
        'armor_class': 12,
        'strength_save_bonus': 1,
        'dexterity_save_bonus': 0,
        'constitution_save_bonus': 2,
        'intelligence_save_bonus': 1,
        'wisdom_save_bonus': 2,
        'charisma_save_bonus': 2,
    },
}
```

### Control

These nodes are used to determine how a tree is traversed. 

* if statement
* and
* or

```python
{
    'node': 'Control',
    'type': 'If',
    'condition': 'expression to evaluate',
    'results': {
        'true': AttackNode(),
        'false': EmptyNode(),
    }
}
```

Alternatively, treat this like a d20test node

```python
{
    'node': 'Control',
    'type': 'If',
    'variables': {
        'answer': 'target.form',
    },
    'results': {
        'wolf': AttackNode(),
        'humanoid': EmptyNode(),
    }
}
```

```python
{
    'node': 'Control',
    'type': 'And',
    'results': [
        Node(),
        Node(),
    ]
}
```

```python
{
    'node': 'Control',
    'type': 'Or',
    'results': [
        Node(),
        Node(),
    ]
}
```

## Examples

### Attack with damage

> _**Fist.** Melee Attack Roll:_ +5, reach 5 ft. _Hit:_ 5 (1d4 + 3) Bludgeoning damage.

An attack roll is made and damage is dealt on a hit or critical hit.

As a crude example (subject to change),

```python
{
    'node': 'Attack',
    'variables:' {
        'd20': RollNode('d20'),
        'mod': 5,
    },
    'results': {
        'crit miss': DamageNode('0', 'Bludgeoning'),
        'miss':      DamageNode('0', 'Bludgeoning'),
        'hit':       DamageNode('1d4 + 3', 'Bludgeoning'),
        'crit hit':  DamageNode('2d4 + 3', 'Bludgeoning'),
    }
}
```

### Attack with damage and condition

> _**Claw.** Melee Attack Roll:_ +5, reach 5 ft. *Hit:* 5 (1d4 + 3) Slashing damage, and the target has the Prone condition.

An attack roll is made, on a hit damage is dealt and the target gains a condition.

As a crude example (subject to change),

```python
{
    'node': 'Attack',
    'variables:' {
        'd20': RollNode('d20'),
        'mod': 5,
    },
    'results': {
        'crit miss': [DamageNode('0', 'Slashing')],
        'miss':      [DamageNode('0', 'Slashing')],
        'hit':       [DamageNode('1d4 + 3', 'Slashing'), ConditionNode('Prone')],
        'crit hit':  [DamageNode('2d4 + 3', 'Slashing'), ConditionNode('Prone')],
    }
}
```

### Attack with damage and condition if criteria is met

> _**Bite.** Melee Attack Roll:_ +4, reach 5 ft. _Hit:_ 6 (1d8 + 2) Piercing damage. If the target is Medium or smaller, it has the Grappled condition (escape DC 12).

An attack roll is made, on a hit damage is dealt and if the target qualifies a condition as well.

As a crude example (subject to change),

```python
{
    'node': 'Attack',
    'variables:' {
        'd20': RollNode('d20'),
        'mod': 4,
    },
    'results': {
        'crit miss': [DamageNode('0', 'Piercing')],
        'miss':      [DamageNode('0', 'Piercing')],
        'hit':       [DamageNode('1d8 + 2', 'Piercing'), IfNode("target.size <= 'Medium'", ConditionNode('Prone'))],
        'crit hit':  [DamageNode('2d8 + 2', 'Piercing'), IfNode("target.size <= 'Medium'", ConditionNode('Prone'))],
    }
}
```

### Save with damage

> _**Trample.** Dexterity Saving Throw:_ DC 16, one creature within 5 feet that has the Prone condition. _Failure:_ 17 (2d10 + 6) Bludgeoning damage. _Success:_ Half damage.

A saving throw is made, on a failure the target takes damage and on a success it takes half damage.

As a crude example (subject to change),

```python
{
    'node': 'Save',
    'variables:' {
        'd20': RollNode('d20'),
        'DC': 16,
    },
    'results': {
        'failure': [DamageNode('2d10 + 6', 'Bludgeoning')],
        'success': [DamageNode('(2d10 + 6)/2', 'Bludgeoning')],
    }
}
```

### Save with damage and condition

> _**Constrict.** Strength Saving Throw:_ DC 12, one Medium or smaller creature the snake can see within 5 feet. _Failure:_ 7 (3d4) Bludgeoning damage, and the target has the Grappled condition (escape DC 12).

A saving throw is made, on a failure the target takes damage and gains a condition.